# Raster to Vector Conversion
This code uses [Dynamic World](https://https://developers.google.com/earth-engine/datasets/catalog/GOOGLE_DYNAMICWORLD_V1) data over Delhi region, reclassify the data, and convert the raster data to a vector data.

In [1]:
import geemap
import ee
ee.Authenticate()
ee.Initialize(project = 'ProjectName') # change the project name accordingly

In [3]:
aoi = ee.FeatureCollection('ProjectName/Asset_name') # select the area of interest from assets
Map = geemap.Map()
Map.centerObject(aoi, 10)
Map.addLayer(aoi, {}, 'Area of Interest')
Map

Map(center=[23.08673573643569, 72.55937593062299], controls=(WidgetControl(options=['position', 'transparent_b…

In [4]:
DW_LULC = (ee.ImageCollection('GOOGLE/DYNAMICWORLD/V1')
    .filterDate('2023-01-01', '2023-12-31')
    .select('label')
    .filterBounds(aoi)
    .mode())

In [5]:
# ['water' : Waterbody
# 'trees': Forest
# 'grass':Wasteland
# 'flooded_vegetation':Wasteland
# 'crops':Cropland
# 'shrub_and_scrub':Wasteland
# 'built': Built-up
# 'bare':Wasteland
# 'snow_and_ice': Wasteland]

# A list of pixel values to replace.
from_list = [0, 1, 2, 3, 4, 5, 6, 7, 8] # Original Class labels
to_list = [1, 2, 3, 3, 4, 3, 5, 4, 4] # Reclassified Class labels

# Reclassified Image
img_remap = DW_LULC.clip(aoi).remap(from_list, to_list,  bandName='label',)

In [6]:
# Mapping of the reclassified image

CLASS_NAMES = [
    'water',
    'forest',
    'crops',
    'built',
    'bare',
]

VIS_PALETTE = [
    '419bdf',
    '397d49',
    '88b053',
    '7a87c6',
    'e49635',
]

# Create an RGB image of the label (most likely class) on [0, 1].
dw_rgb = (
    img_remap.select('remapped')
    .visualize(min=1, max=5, palette=VIS_PALETTE)
    .divide(255)
)

In [7]:
Map.addLayer(dw_rgb, {}, "Reclassify")
Map

Map(bottom=114086.0, center=[23.08673573643569, 72.55937593062299], controls=(WidgetControl(options=['position…

In [10]:
# conversion of raster to vector
vectors = img_remap.reduceToVectors(
  geometry = aoi,
  crs = DW_LULC.projection(),
  scale= 10,
  geometryType= 'polygon',
  eightConnected = False,
  labelProperty = 'zone',
  maxPixels = 1e9
);

In [11]:
Map.addLayer(vectors, {}, 'vectors')
Map

Map(bottom=114086.0, center=[23.08673573643569, 72.55937593062299], controls=(WidgetControl(options=['position…

In [ ]:
# Export the data to google drive
geemap.ee_export_vector_to_drive(vectors, description="FileName", folder="FolderName",fileFormat='SHP')